In [7]:
from rosbag2_reader_py import Rosbag2Reader

## Open ROS 2 Bag

**The file `rosbag2_reader_py.py` shall be in the same folder of the notebook.**

In [8]:
path = "/home/federico/lecture_ws/rosbags_lab04/rosbag2_2024_11_22-19_58_02"

reader = Rosbag2Reader(path)
topics = reader.all_topics
topics

[INFO] [1732568818.515260790] [rosbag2_storage]: Opened database '/home/federico/lecture_ws/rosbags_lab04/rosbag2_2024_11_22-19_58_02/rosbag2_2024_11_22-19_58_02_0.db3' for READ_ONLY.
[INFO] [1732568818.516483538] [rosbag2_storage]: Opened database '/home/federico/lecture_ws/rosbags_lab04/rosbag2_2024_11_22-19_58_02/rosbag2_2024_11_22-19_58_02_0.db3' for READ_ONLY.


{'/ekf': 'nav_msgs/msg/Odometry',
 '/ground_truth': 'nav_msgs/msg/Odometry',
 '/odom': 'nav_msgs/msg/Odometry'}

## Get total number of messages in the bag

In [9]:
tot_msgs = 0
for _ in reader:
    tot_msgs += 1

print(f"Total messages: {tot_msgs}")

[INFO] [1732568825.201415153] [rosbag2_storage]: Opened database '/home/federico/lecture_ws/rosbags_lab04/rosbag2_2024_11_22-19_58_02/rosbag2_2024_11_22-19_58_02_0.db3' for READ_ONLY.


Total messages: 5542


## Select messages of specific topics

In [10]:
tot_msgs = 0
reader.set_filter(["/odom"])
for _ in reader:
    tot_msgs += 1

print("After the filter is applyed: ", reader.selected_topics)
print(f"Total messages: {tot_msgs}")

reader.reset_filter() # if you want to read all messages after you set a filter
print("After the filter is reset: ", reader.selected_topics)

[INFO] [1732568836.874218457] [rosbag2_storage]: Opened database '/home/federico/lecture_ws/rosbags_lab04/rosbag2_2024_11_22-19_58_02/rosbag2_2024_11_22-19_58_02_0.db3' for READ_ONLY.
[INFO] [1732568836.875577327] [rosbag2_storage]: Opened database '/home/federico/lecture_ws/rosbags_lab04/rosbag2_2024_11_22-19_58_02/rosbag2_2024_11_22-19_58_02_0.db3' for READ_ONLY.


After the filter is applyed:  {'/odom': 'nav_msgs/msg/Odometry'}
Total messages: 2582
After the filter is reset:  {'/ekf': 'nav_msgs/msg/Odometry', '/ground_truth': 'nav_msgs/msg/Odometry', '/odom': 'nav_msgs/msg/Odometry'}


[INFO] [1732568837.300235975] [rosbag2_storage]: Opened database '/home/federico/lecture_ws/rosbags_lab04/rosbag2_2024_11_22-19_58_02/rosbag2_2024_11_22-19_58_02_0.db3' for READ_ONLY.


## Access only data of a given type

In this example you can see  how to access an `Odometry` message checking for its type.

Please, notice the difference between **recording time** and time reported in the **stamp**. This is because the message was generated at a time that does not coincide with the time the message was received and recorded. This difference in a real robot may be really small, in the order of microseconds, but for a simulation, as in the reported case, the time could be extremely different. **You shall always use `header.stamp` whenever it is available.**

In [11]:
from rclpy.time import Time
from nav_msgs.msg import Odometry
for topic_name, msg, t in reader:
    print(f"Received message of type {type(msg).__name__} on topic {topic_name} recorded at time {t}")
    if type(msg) is Odometry:
        time = Time.from_msg(msg.header.stamp).nanoseconds
        print(f"Position (x, y) at time {time}: ({msg.pose.pose.position.x:.2f}, {msg.pose.pose.position.y:.2f})")
    break

Received message of type Odometry on topic /odom recorded at time 1732301882677025509
Position (x, y) at time 20085000000: (-2.00, -0.50)


[INFO] [1732568850.412180206] [rosbag2_storage]: Opened database '/home/federico/lecture_ws/rosbags_lab04/rosbag2_2024_11_22-19_58_02/rosbag2_2024_11_22-19_58_02_0.db3' for READ_ONLY.


## Interpolate data to compute metrics

In order to compute the metrics for `/odom` and `/ekf` topics, you have to compare the poses reported in these topic with the poses reported in topic `/ground_truth` in the same time instants. 

Since the data are generated from different nodes at different frequencies, the time of the various topics will be different. So, we need to interpolate ground truth data on the time scale of the topic we want to evaluate.

First of all, let us save relevant data from messages in some NumPy arrays. As you can see from the output, the number of points from the two topics is different.

In [15]:
import numpy as np
from scipy.interpolate import interp1d

time_gt = []
gt_data = []
time_odom = []
odom_data = []

for topic_name, msg, t in reader:
    if topic_name == "/ground_truth":
        time_gt.append(Time.from_msg(msg.header.stamp).nanoseconds)
        gt_data.append((msg.pose.pose.position.x, msg.pose.pose.position.y))
    elif topic_name == "/odom":
        time_odom.append(Time.from_msg(msg.header.stamp).nanoseconds)
        odom_data.append((msg.pose.pose.position.x, msg.pose.pose.position.y))

time_gt = np.array(time_gt)
gt_data = np.array(gt_data)
time_odom = np.array(time_odom)
odom_data = np.array(odom_data)

print(f"Ground truth points: {len(gt_data)}")
print(f"Odometry points: {len(odom_data)}")

[INFO] [1732569055.923755094] [rosbag2_storage]: Opened database '/home/federico/lecture_ws/rosbags_lab04/rosbag2_2024_11_22-19_58_02/rosbag2_2024_11_22-19_58_02_0.db3' for READ_ONLY.


Ground truth points: 1756
Odometry points: 2582


Now, let us create an interpolating function using SciPy `interp1d`.

In [16]:
gt_interpol = interp1d(time_gt, gt_data, axis=0, fill_value="extrapolate")
gt_data_interp = gt_interpol(time_odom)
print(f"Interpolated ground truth points: {len(gt_data_interp)}")

Interpolated ground truth points: 2582


Compute Mean Absolute Error between odometry data and interpolated ground truth. You can find already implemented metrics functionson Portale della Didattica (Lecture_notebooks/Gaussian_filters.zip/utils.py)

In [17]:
np.mean(np.linalg.norm(odom_data - gt_data_interp, axis=1), axis=0)

np.float64(3.189979817822099e-06)

In [3]:
################ Task2


from rosbag2_reader_py import Rosbag2Reader
from rclpy.time import Time
from nav_msgs.msg import Odometry
from geometry_msgs.msg import PoseStamped
import numpy as np
from scipy.interpolate import interp1d


def _error(actual: np.ndarray, predicted: np.ndarray):
    """ Simple error """
    return actual - predicted

def mse(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Squared Error """
    if len(actual.shape)==1 and len(predicted.shape)==1:
        return np.mean(np.square(_error(actual, predicted)), axis=0)
    return np.mean(np.sum(np.square(_error(actual, predicted)), axis=1), axis=0)

def rmse(actual: np.ndarray, predicted: np.ndarray):
    """ Root Mean Squared Error """
    return np.sqrt(mse(actual, predicted))

def mae(error: np.ndarray):
    """ Mean Absolute Error """
    return np.mean(np.abs(error))

path = "/home/federico/lecture_ws/rosbags_lab04/rosbag2_2024_11_22-19_58_02"

reader = Rosbag2Reader(path)

time_gt = []
gt_data = []
time_odom = []
odom_data = []
time_ekf = []
ekf_data = []

for topic_name, msg, t in reader:
    if topic_name == "/ground_truth":
        time_gt.append(Time.from_msg(msg.header.stamp).nanoseconds)
        gt_data.append((msg.pose.pose.position.x, msg.pose.pose.position.y))
    elif topic_name == "/odom":
        time_odom.append(Time.from_msg(msg.header.stamp).nanoseconds)
        odom_data.append((msg.pose.pose.position.x, msg.pose.pose.position.y))
    elif topic_name == "/ekf":
        time_ekf.append(Time.from_msg(msg.header.stamp).nanoseconds)
        ekf_data.append((msg.pose.pose.position.x, msg.pose.pose.position.y))

time_gt = np.array(time_gt)
gt_data = np.array(gt_data)
time_odom = np.array(time_odom)
odom_data = np.array(odom_data)
time_ekf = np.array(time_ekf)
ekf_data = np.array(ekf_data)

gt_interpol = interp1d(time_gt, gt_data, axis=0, fill_value="extrapolate")
gt_data_interp_odom = gt_interpol(time_odom)
gt_data_interp_ekf = gt_interpol(time_ekf)

error_odom = gt_data_interp_odom - odom_data
error_ekf = gt_data_interp_ekf - ekf_data

rmse_odom = rmse(gt_data_interp_odom, odom_data)
mae_odom = mae(error_odom)
rmse_ekf = rmse(gt_data_interp_ekf, ekf_data)
mae_ekf = mae(error_ekf)

rmse_odom_scalar = np.mean(rmse_odom)
mae_odom_scalar = np.mean(mae_odom)
rmse_ekf_scalar = np.mean(rmse_ekf)
mae_ekf_scalar = np.mean(mae_ekf)

# Print results
print(f"RMSE between Ground Truth and Odometry: {rmse_odom_scalar:.2f}")
print(f"MAE between Ground Truth and Odometry: {mae_odom_scalar:.2f}")
print(f"RMSE between Ground Truth and EKF: {rmse_ekf_scalar:.2f}")
print(f"MAE between Ground Truth and EKF: {mae_ekf_scalar:.2f}")


[INFO] [1732616230.218392840] [rosbag2_storage]: Opened database '/home/federico/lecture_ws/rosbags_lab04/rosbag2_2024_11_22-19_58_02/rosbag2_2024_11_22-19_58_02_0.db3' for READ_ONLY.
[INFO] [1732616230.219418579] [rosbag2_storage]: Opened database '/home/federico/lecture_ws/rosbags_lab04/rosbag2_2024_11_22-19_58_02/rosbag2_2024_11_22-19_58_02_0.db3' for READ_ONLY.
[INFO] [1732616230.221084585] [rosbag2_storage]: Opened database '/home/federico/lecture_ws/rosbags_lab04/rosbag2_2024_11_22-19_58_02/rosbag2_2024_11_22-19_58_02_0.db3' for READ_ONLY.


RMSE between Ground Truth and Odometry: 0.00
MAE between Ground Truth and Odometry: 0.00
RMSE between Ground Truth and EKF: 8804.62
MAE between Ground Truth and EKF: 5890.90


In [4]:
# Normalize timestamps for both Ground Truth and EKF
time_gt_norm = time_gt - np.min(time_gt)  # Normalize Ground Truth timestamps
time_ekf_norm = time_ekf - np.min(time_ekf)  # Normalize EKF timestamps

gt_interpol = interp1d(time_gt_norm, gt_data, axis=0, fill_value="extrapolate")
gt_data_interp_ekf = gt_interpol(time_ekf_norm)

error_ekf = gt_data_interp_ekf - ekf_data

rmse_ekf = rmse(gt_data_interp_ekf, ekf_data)
mae_ekf = mae(error_ekf)

print(f"RMSE between Ground Truth and EKF: {rmse_ekf:.2f}")
print(f"MAE between Ground Truth and EKF: {mae_ekf:.2f}")

RMSE between Ground Truth and EKF: 0.15
MAE between Ground Truth and EKF: 0.06


In [11]:
###################Task1 odom

from rosbag2_reader_py import Rosbag2Reader
from rclpy.time import Time
from nav_msgs.msg import Odometry
from geometry_msgs.msg import PoseStamped
import numpy as np
from scipy.interpolate import interp1d

def mse(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Squared Error """
    if len(actual.shape)==1 and len(predicted.shape)==1:
        return np.mean(np.square(_error(actual, predicted)), axis=0)
    return np.mean(np.sum(np.square(_error(actual, predicted)), axis=1), axis=0)

def rmse(actual: np.ndarray, predicted: np.ndarray):
    """ Root Mean Squared Error """
    return np.sqrt(mse(actual, predicted))

def mae(error: np.ndarray):
    """ Mean Absolute Error """
    return np.mean(np.abs(error))

path = "/home/federico/lecture_ws/rosbags_lab04/rosbag2_2024_11_22-17_32_28"

reader = Rosbag2Reader(path)

time_gt = []
gt_data = []
time_odom = []
odom_data = []
time_ekf = []
ekf_data = []

for topic_name, msg, t in reader:
    if topic_name == "/ground_truth":
        time_gt.append(Time.from_msg(msg.header.stamp).nanoseconds)
        gt_data.append((msg.pose.pose.position.x, msg.pose.pose.position.y))
    elif topic_name == "/odom":
        time_odom.append(Time.from_msg(msg.header.stamp).nanoseconds)
        odom_data.append((msg.pose.pose.position.x, msg.pose.pose.position.y))
    elif topic_name == "/ekf":
        time_ekf.append(Time.from_msg(msg.header.stamp).nanoseconds)
        ekf_data.append((msg.pose.pose.position.x, msg.pose.pose.position.y))

time_gt = np.array(time_gt)
gt_data = np.array(gt_data)
time_odom = np.array(time_odom)
odom_data = np.array(odom_data)
time_ekf = np.array(time_ekf)
ekf_data = np.array(ekf_data)

# Normalize timestamps for both Ground Truth and EKF
time_gt_norm = time_gt - np.min(time_gt)  # Normalize Ground Truth timestamps
time_ekf_norm = time_ekf - np.min(time_ekf)  # Normalize EKF timestamps
time_odom_norm = time_odom - np.min(time_odom)

gt_interpol = interp1d(time_gt_norm, gt_data, axis=0, fill_value="extrapolate")
gt_data_interp_ekf = gt_interpol(time_ekf_norm)

error_ekf = gt_data_interp_ekf - ekf_data
gt_data_interp_odom = gt_interpol(time_odom_norm)
gt_data_interp_ekf = gt_interpol(time_ekf_norm)

error_odom = gt_data_interp_odom - odom_data
error_ekf = gt_data_interp_ekf - ekf_data

rmse_odom = rmse(gt_data_interp_odom, odom_data)
mae_odom = mae(error_odom)
rmse_ekf = rmse(gt_data_interp_ekf, ekf_data)
mae_ekf = mae(error_ekf)

rmse_odom_scalar = np.mean(rmse_odom)
mae_odom_scalar = np.mean(mae_odom)
rmse_ekf_scalar = np.mean(rmse_ekf)
mae_ekf_scalar = np.mean(mae_ekf)

# Print results
print(f"RMSE between Ground Truth and Odometry: {rmse_odom_scalar:.2f}")
print(f"MAE between Ground Truth and Odometry: {mae_odom_scalar:.2f}")
print(f"RMSE between Ground Truth and EKF: {rmse_ekf_scalar:.2f}")
print(f"MAE between Ground Truth and EKF: {mae_ekf_scalar:.2f}")


[INFO] [1732617843.795087086] [rosbag2_storage]: Opened database '/home/federico/lecture_ws/rosbags_lab04/rosbag2_2024_11_22-17_32_28/rosbag2_2024_11_22-17_32_28_0.db3' for READ_ONLY.
[INFO] [1732617843.796741901] [rosbag2_storage]: Opened database '/home/federico/lecture_ws/rosbags_lab04/rosbag2_2024_11_22-17_32_28/rosbag2_2024_11_22-17_32_28_0.db3' for READ_ONLY.
[INFO] [1732617843.798273487] [rosbag2_storage]: Opened database '/home/federico/lecture_ws/rosbags_lab04/rosbag2_2024_11_22-17_32_28/rosbag2_2024_11_22-17_32_28_0.db3' for READ_ONLY.


RMSE between Ground Truth and Odometry: 0.00
MAE between Ground Truth and Odometry: 0.00
RMSE between Ground Truth and EKF: 0.54
MAE between Ground Truth and EKF: 0.24


In [12]:
###################Task1 cdm

from rosbag2_reader_py import Rosbag2Reader
from rclpy.time import Time
from nav_msgs.msg import Odometry
from geometry_msgs.msg import PoseStamped
import numpy as np
from scipy.interpolate import interp1d

def mse(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Squared Error """
    if len(actual.shape)==1 and len(predicted.shape)==1:
        return np.mean(np.square(_error(actual, predicted)), axis=0)
    return np.mean(np.sum(np.square(_error(actual, predicted)), axis=1), axis=0)

def rmse(actual: np.ndarray, predicted: np.ndarray):
    """ Root Mean Squared Error """
    return np.sqrt(mse(actual, predicted))

def mae(error: np.ndarray):
    """ Mean Absolute Error """
    return np.mean(np.abs(error))

path = "/home/federico/lecture_ws/rosbags_lab04/sim_task1_cdm/rosbag2_2024_11_20-22_02_07"

reader = Rosbag2Reader(path)

time_gt = []
gt_data = []
time_odom = []
odom_data = []
time_ekf = []
ekf_data = []

for topic_name, msg, t in reader:
    if topic_name == "/ground_truth":
        time_gt.append(Time.from_msg(msg.header.stamp).nanoseconds)
        gt_data.append((msg.pose.pose.position.x, msg.pose.pose.position.y))
    elif topic_name == "/odom":
        time_odom.append(Time.from_msg(msg.header.stamp).nanoseconds)
        odom_data.append((msg.pose.pose.position.x, msg.pose.pose.position.y))
    elif topic_name == "/ekf":
        time_ekf.append(Time.from_msg(msg.header.stamp).nanoseconds)
        ekf_data.append((msg.pose.pose.position.x, msg.pose.pose.position.y))

time_gt = np.array(time_gt)
gt_data = np.array(gt_data)
time_odom = np.array(time_odom)
odom_data = np.array(odom_data)
time_ekf = np.array(time_ekf)
ekf_data = np.array(ekf_data)

# Normalize timestamps for both Ground Truth and EKF
time_gt_norm = time_gt - np.min(time_gt)  # Normalize Ground Truth timestamps
time_ekf_norm = time_ekf - np.min(time_ekf)  # Normalize EKF timestamps
time_odom_norm = time_odom - np.min(time_odom)

gt_interpol = interp1d(time_gt_norm, gt_data, axis=0, fill_value="extrapolate")
gt_data_interp_ekf = gt_interpol(time_ekf_norm)

error_ekf = gt_data_interp_ekf - ekf_data
gt_data_interp_odom = gt_interpol(time_odom_norm)
gt_data_interp_ekf = gt_interpol(time_ekf_norm)

error_odom = gt_data_interp_odom - odom_data
error_ekf = gt_data_interp_ekf - ekf_data

rmse_odom = rmse(gt_data_interp_odom, odom_data)
mae_odom = mae(error_odom)
rmse_ekf = rmse(gt_data_interp_ekf, ekf_data)
mae_ekf = mae(error_ekf)

rmse_odom_scalar = np.mean(rmse_odom)
mae_odom_scalar = np.mean(mae_odom)
rmse_ekf_scalar = np.mean(rmse_ekf)
mae_ekf_scalar = np.mean(mae_ekf)

# Print results
print(f"RMSE between Ground Truth and Odometry: {rmse_odom_scalar:.2f}")
print(f"MAE between Ground Truth and Odometry: {mae_odom_scalar:.2f}")
print(f"RMSE between Ground Truth and EKF: {rmse_ekf_scalar:.2f}")
print(f"MAE between Ground Truth and EKF: {mae_ekf_scalar:.2f}")


[INFO] [1732636279.609096439] [rosbag2_storage]: Opened database '/home/federico/lecture_ws/rosbags_lab04/sim_task1_cdm/rosbag2_2024_11_20-22_02_07/rosbag2_2024_11_20-22_02_07_0.db3' for READ_ONLY.
[INFO] [1732636279.610384591] [rosbag2_storage]: Opened database '/home/federico/lecture_ws/rosbags_lab04/sim_task1_cdm/rosbag2_2024_11_20-22_02_07/rosbag2_2024_11_20-22_02_07_0.db3' for READ_ONLY.
[INFO] [1732636279.613174415] [rosbag2_storage]: Opened database '/home/federico/lecture_ws/rosbags_lab04/sim_task1_cdm/rosbag2_2024_11_20-22_02_07/rosbag2_2024_11_20-22_02_07_0.db3' for READ_ONLY.


RMSE between Ground Truth and Odometry: 0.00
MAE between Ground Truth and Odometry: 0.00
RMSE between Ground Truth and EKF: 0.19
MAE between Ground Truth and EKF: 0.08
